In [ ]:
import sys
from os import path
from os.path import exists
import text2emotion as te
import pandas as pd
import json
from collections import Counter
import matplotlib.pyplot as plt
import sagemaker
import boto3

In [ ]:
role = sagemaker.get_execution_role()

s3 = boto3.resource('s3')

bucket_name = 'ad440-mpg-floop-export-storage'

obj = s3.Object(bucket_name, 'auto-floop-s3-export3-sagemaker.json')

s3_client = boto3.client('s3')

data = json.load(obj.get()['Body'])
print(data[0])

In [ ]:
# Get few lines from file for sample
dataList=[]
for i in range(10):
    dataList.append(data[i])

In [ ]:
# Create array of the sample sentences to analyze
sentences=[]

# switch dataList to data for get all data
for x in dataList:
    for y in x:       
        sentences.append(y['Text'])
print(sentences)

In [ ]:
def isEmotion(sentence):
    emotions = te.get_emotion(sentence)
    max_val = max(emotions.values())
    if max_val == 0:
        return 'can\'t determine emotion'
    else:
        # return highest value emotion
        emotion = max(emotions, key=lambda key: emotions[key])
        return emotion

In [ ]:
# Determine emotion for each sentence
emotionList=[]
for i in sentences:
    emotionList.append(isEmotion(i))
    
countEmotions = Counter(emotionList)

In [ ]:
# Data frame for original sentence and emotion analyzed
df = pd.DataFrame(list(zip(sentences, emotionList)),columns=['Sentence', 'Emotion'])
df

In [ ]:
df.to_csv('sentenceEmotion.csv')

In [ ]:
fig, axl = plt.subplots()
axl.bar(countEmotions.keys(), countEmotions.values())
fig.autofmt_xdate()
plt.show()